In [1]:
import numpy as np
import pandas as pd
import obspy
from obspy.taup import TauPyModel
from scipy.stats import norm
import scipy.interpolate as spi
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
df_mars = pd.read_csv("Downloads/modified_mars_seismic_data_m_005.csv")


mars_velocity_model = {
    "depth_km": [0, 400, 800, 1200, 1600, 1800, 2000, 2200, 2500, 2700, 2900, 3200],
    "p_wave_velocity_km_s": [5.0, 6.0, 7.5, 8.2, 8.8, 9.2, 9.5, 9.7, 10.0, 10.2, 10.4, 10.5],
    "s_wave_velocity_km_s": [3.0, 3.5, 4.2, 4.6, 4.9, 5.1, 5.3, 5.4, 5.5, 5.6, 5.6, 0.0],
    "density_g_cm3": [3.5, 3.6, 3.7, 3.8, 4.0, 4.2, 4.5, 4.7, 5.0, 5.2, 5.4, 5.6]
}


mars_depths = np.array(mars_velocity_model["depth_km"])
mars_p_velocity = np.array(mars_velocity_model["p_wave_velocity_km_s"])
mars_s_velocity = np.array(mars_velocity_model["s_wave_velocity_km_s"])
mars_density = np.array(mars_velocity_model["density_g_cm3"])
mars_p_interp = spi.interp1d(mars_depths, mars_p_velocity, kind="cubic", fill_value="extrapolate")
mars_s_interp = spi.interp1d(mars_depths, mars_s_velocity, kind="cubic", fill_value="extrapolate")
mars_density_interp = spi.interp1d(mars_depths, mars_density, kind="cubic", fill_value="extrapolate")

In [3]:
def seismic_likelihood_advanced(depth, core_radius, sigma=0.1):
    
    expected_s_wave = mars_s_interp(depth)
    expected_density = mars_density_interp(depth)
    
    if expected_s_wave <= 0.1 and expected_density > 5.0:
        return np.exp(-((expected_density / core_radius) ** 2) / (2 * sigma**2))
    
    return 0.1

In [4]:
depth_samples = np.linspace(1000, 2500, 100)
posterior_probs_advanced = np.array([seismic_likelihood_advanced(d, 1700) for d in depth_samples])


bayesian_core_radius_advanced = depth_samples[np.argmax(posterior_probs_advanced)]


likelihoods = norm.pdf(depth_samples, loc=bayesian_core_radius_advanced, scale=100)


posterior_final = posterior_probs_advanced * likelihoods
posterior_final /= posterior_final.sum()  

In [5]:
final_core_radius_bayesian_advanced = depth_samples[np.argmax(posterior_final)]


mars_radius = 3389.5  
mars_core_radius_final = mars_radius - final_core_radius_bayesian_advanced

print(f"Estimated Core Radius of Mars (Bayesian Inference): {mars_core_radius_final:.2f} km")


mars_core_radius_label = np.full(len(df_mars), mars_core_radius_final)  

features = [
     "P-wave Velocity (km/s)", "S-wave Velocity (km/s)",
    "Density (g/cm^3)"
]

Estimated Core Radius of Mars (Bayesian Inference): 2389.50 km
